In [87]:
import pandas as pd
import numpy as np
from scipy import linalg

In [2]:
import os
os.chdir("../")

In [12]:
from comchoice.datasets import load_synthetic_pairwise
from comchoice.pairwise import Pairwise

In [18]:
df = load_synthetic_pairwise(n_voters=4, n_alternatives=5)
choice = Pairwise(df)
choice.fit()

choice.df.head()

,voter,option_a,option_b,selected,alternative_a_sorted,alternative_b_sorted,option_selected,card_id,option_source,option_target,id
0,1,5,1,1,1,5,1,1_5,5,1,1
1,1,2,1,1,1,2,1,1_2,2,1,2
2,1,3,1,1,1,3,1,1_3,3,1,3
3,1,1,4,1,1,4,1,1_4,4,1,4
4,1,2,5,5,2,5,-1,2_5,2,5,5


In [19]:
ahp(choice.df)

,alternative,value
0,1,0.353404
1,2,0.230246
2,3,0.157801
3,4,0.060749
4,5,0.197801


In [80]:
S, D, Vt = np.linalg.svd(cc, full_matrices=False)

In [101]:
c = cc / cc.sum()
weight = [1/c.shape[0] for i in range(c.shape[0])]
weight

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [195]:
dd = np.array([[1, 0.25, 4], [4, 1, 9], [0.25, 0.11111111, 1]])

u, s, vT = np.linalg.svd(cc)
u[0]

array([-0.92015262, -0.39151407, -0.00599046])

In [145]:
cc.sum()

option_b
Harry     1.361111
Dick     14.000000
Tom       5.250000
dtype: float64

In [154]:

(cc / cc.sum(axis=0)).mean(axis=1)

option_a
Harry    0.713152
Dick     0.066893
Tom      0.219955
dtype: float64

In [198]:
n = 3

priority="eigval"

_lambda = linalg.eigvals(cc)[0].real
_lambda = np.multiply(cc.sum(axis=0), (cc / cc.sum(axis=0)).mean(axis=1)).sum()

# Random Consistency Index
random_index = {
    1: 0, 2: 0, 3: 0.52, 4: 0.89, 5: 1.11, 6: 1.25, 7: 1.35,
    8: 1.4, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.54, 13: 1.56, 14: 1.57, 15: 1.58
}

# Calculates Consistency Index
ci = (_lambda - n) / (n - 1)

# Calculates Consistency Ratio
cr = ci / random_index[n]
ci, cr

(0.03097442680776008, 0.059566205399538615)

In [262]:
def ahp(df, ppal_eigval="approximation", criteria="criteria"):
    
    def __calc(df, ppal_eigval=ppal_eigval):
        df["value"] = df["weight_b"] / df["weight_a"]

        items = set(df["option_a"]) | set(df["option_b"])
        n = len(items)

        tmp = df.pivot(index="option_b", columns="option_a", values="value")
        tmp = tmp.reindex(items, axis=0)
        tmp = tmp.reindex(items, axis=1)

        tmp = tmp.fillna(0) + (1 / tmp.T).fillna(0)
        np.fill_diagonal(tmp.values, 1)

        sum_cols = tmp.sum(axis=0)
        weight = tmp / tmp.sum(axis=0)
        priority = weight.mean(axis=1)

        if ppal_eigval == "eigval":
            _lambda = linalg.eigvals(cc)[0].real
        elif ppal_eigval == "approximation":
            _lambda = np.multiply(sum_cols, priority).sum()
        else:
            raise "Value provided to ppal_eigval parameter not valid. Values accepted are 'eigval', 'approximation'"

        priority = pd.DataFrame(priority).reset_index()
        priority.columns = ["option", "value"]

        # Calculates Consistency Index
        ci = (_lambda - n) / (n - 1)

        # Calculates Consistency Ratio
        random_index = {
            1: 0, 2: 0, 3: 0.52, 4: 0.89, 5: 1.11, 6: 1.25, 7: 1.35,
            8: 1.4, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.54, 13: 1.56, 14: 1.57, 15: 1.58
        } # Random Consistency Index

        cr = ci / random_index[n]

        return priority, ci, cr
    
    
    if criteria in list(df):
        output = []
        for i, df_tmp in df.groupby(criteria):
            priority, ci, cr = __calc(df_tmp, ppal_eigval=ppal_eigval)
            priority[criteria] = i
            output.append(priority)
        
        df_output = pd.concat(output)
        df_output = df_output.reset_index()
        return df_output, ci, cr
        
    priority, ci, cr = __calc(df, ppal_eigval=ppal_eigval)
    
    return priority, ci, cr

In [258]:
# Alternatives vs Criteria
aa = pd.DataFrame([
    ("Tom", "Dick", 1, 4, "Experience"),
    ("Tom", "Harry", 4, 1, "Experience"),
    ("Dick", "Harry", 9, 1, "Experience"),
    ("Tom", "Dick", 3, 1, "Education"),
    ("Tom", "Harry", 1, 5, "Education"),
    ("Dick", "Harry", 1, 7, "Education"),
    ("Tom", "Dick", 3, 1, "Charisma"),
    ("Tom", "Harry", 9, 1, "Charisma"),
    ("Dick", "Harry", 4, 1, "Charisma"),
    ("Tom", "Dick", 1, 3, "Age"),
    ("Tom", "Harry", 5, 1, "Age"),
    ("Dick", "Harry", 9, 1, "Age"),
], columns=["option_a", "option_b", "weight_a", "weight_b", "criteria"])

ahp(aa)

,option,value,criteria
0,Harry,0.063736,Age
1,Dick,0.668864,Age
2,Tom,0.267399,Age
0,Harry,0.069058,Charisma
1,Dick,0.250593,Charisma
2,Tom,0.680349,Charisma
0,Harry,0.723506,Education
1,Dick,0.083308,Education
2,Tom,0.193186,Education
0,Harry,0.066893,Experience


In [255]:
# Criteria vs Goal

In [259]:
bb = pd.DataFrame([
    ("Experience", "Education", 4, 1),
    ("Experience", "Charisma", 3, 1),
    ("Experience", "Age", 7, 1),
    ("Education", "Charisma", 1, 3),
    ("Education", "Age", 3, 1),
    ("Age", "Charisma", 1, 5)
], columns=["option_a", "option_b", "weight_a", "weight_b"])

ahp(bb)

,option,value
0,Experience,0.539644
1,Education,0.131464
2,Charisma,0.271548
3,Age,0.057344


In [261]:
cc = pd.merge(ahp(aa), ahp(bb), left_on="criteria", right_on="option")
cc["value"] = cc["value_x"] * cc["value_y"]
cc.groupby("option_x").agg({"value": "sum"})

,value
option_x,
Dick,0.502203
Harry,0.153621
Tom,0.344175


In [10]:
def ahp(df, alternative="alternative", voter="voter") -> pd.DataFrame:

    dd = df.groupby(["option_source", "option_target"]).agg(
        {voter: "count"}).reset_index()

    a = dd.pivot(index="option_target",
                 columns="option_source", values=voter)
    b = np.divide(a, a.T)
    np.fill_diagonal(b.values, 1)
    c = b / b.sum()
    weight = [1/c.shape[0] for i in range(c.shape[0])]

    tmp = pd.DataFrame(np.sum(np.multiply(c, weight), axis=1)).reset_index().rename(
        columns={"option_target": alternative, 0: "value"})

    return tmp